# Stylometry

In [4]:
from expertai.nlapi.cloud.client import ExpertAiClient
client = ExpertAiClient()

In [3]:
pip install expertai-nlapi

     |████████████████████████████████| 97 kB 3.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
os.environ["EAI_USERNAME"] = 'nihaghali554@gmail.com'
os.environ["EAI_PASSWORD"] = 'Daltadka@@21'

In [2]:
import pandas as pd

In [3]:
df= pd.read_csv("speeches/presidential_speeches.csv")

In [4]:
df.head()

,Date,President,Party,Speech Title,Summary,Transcript,URL
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...


In [5]:
import random 
import nltk
from itertools import chain

In [6]:
def chunker(full_text, chunk_size=9000):
    tokenized_text= nltk.tokenize.sent_tokenize(full_text)
    chunked_text=[]
    while len(tokenized_text) >0:
        character_sum=0
        chunk=[]
        counter=0
        while character_sum <= chunk_size:
            try:
                chunk.append(tokenized_text[counter])
                character_sum += len(tokenized_text[counter])
                counter += 1
            except IndexError:
                break
        chunked_text.append(" ".join(chunk))
        tokenized_text = tokenized_text[counter:]
    return chunked_text

In [7]:
 random.seed(1138)

In [2]:
# import nltk
# nltk.download('all')
# nltk.download('punkt')

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
obama_speeches = random.sample(
    list(chain.from_iterable([chunker(speech) for speech in df[df.President=="Barack Obama"].Transcript.to_list()])),60)
trump_speeches = random.sample(
    list(chain.from_iterable([chunker(speech) for speech in df[df.President=="Donald Trump"].Transcript.to_list()])),60)

In [9]:
presidential_speeches = pd.DataFrame({"text": obama_speeches + trump_speeches, "author": ["obama"]*60 + ["trump"]*60 })

In [ ]:
presidential_speeches.head()

# Adding writeprint

In [ ]:
from expertai.nlapi.cloud.client import ExpertAiClient
from dotenv import load_dotenv

In [12]:
def make_one_writeprint(string):
    load_dotenv()
    client = ExpertAiClient()
    return client.detection(
    body={"document": {"text": string}},
    params={"detector": "writeprint","language":"en"})

In [15]:
writeprint_test = make_one_writeprint(presidential_speeches.text[0])
writeprint_test.extra_data

{'JSON-LD': {'@context': {'readabilityIndexes': 'https://schema.org/additionalProperty',
   'name': 'https://schema.org/name',
   'value': 'https://schema.org/value',
   'total': 'https://schema.org/Number',
   'readabilityLevel': 'https://schema.org/Text',
   'structureIndexes': 'https://schema.org/Thing',
   'mean': 'https://schema.org/Number',
   'standardDeviation': 'https://schema.org/Number',
   'meanAbsoluteDeviation': 'https://schema.org/Number',
   'sentences': 'https://schema.org/QuantitativeValue',
   'tokens': 'https://schema.org/QuantitativeValue',
   'tokenLengthPerSentence': 'https://schema.org/QuantitativeValue',
   'verbTypes': 'https://schema.org/QuantitativeValue',
   'charactersPerSentence': 'https://schema.org/QuantitativeValue',
   'atomsPerSentence': 'https://schema.org/QuantitativeValue',
   'tokensPerSentence': 'https://schema.org/QuantitativeValue',
   'phrasesPerSentence': 'https://schema.org/QuantitativeValue',
   'verbTypesPerSentence': 'https://schema.org/

In [16]:
from tqdm import tqdm

In [17]:
def extract_indexes_names(one_wp_obj):
    index_names=[]
    indexes=[]
    for readabilityIndex in one_wp_obj.extra_data["JSON-LD"]["@graph"][0]['readabilityIndexes']:
        index_names.append(readabilityIndex['name'].replace(" ", "").replace("-",""))
        indexes.append(readabilityIndex['value'])
    for structureIndex in one_wp_obj.extra_data["JSON-LD"]["@graph"][0]['structureIndexes']:
        try:
            indexes.append(one_wp_obj.extra_data["JSON-LD"]["@graph"][0]['structureIndexes'][structureIndex]['mean'])
            index_names.append(structureIndex)
        
        except KeyError:
                           pass
        
    return indexes, index_names

In [18]:
def add_wp_values(old_table):
    new_table=[]
    for _ in tqdm(range(len(old_table))):
        text, author = old_table.iloc[_]
        try:
            one_wp = make_one_writeprint(text)
        except:
            print(_)
            continue
        indexes, index_names =extract_indexes_names(one_wp)
        new_row = [text, author] + indexes
        new_table.append(new_row)
    columns=list(old_table. columns) + index_names
    new_df=pd.DataFrame(new_table, columns=columns)
    return new_df

In [20]:
speeches_with_wp=add_wp_values(presidential_speeches)

100%|██████████| 120/120 [02:24<00:00,  1.21s/it]


In [21]:
speeches_with_wp.head()

,text,author,ColemanLiau,Gulpease,AutomatedReadabilityIndex,tokenLengthPerSentence,charactersPerSentence,atomsPerSentence,tokensPerSentence,phrasesPerSentence,...,exclamationMarkQuestionMarkSequencesPerSentence,multipleDotsPerSentence,multipleExclamationMarksPerSentence,multipleQuestionMarksPerSentence,questionMarksPerSentence,semicolonsPerSentence,singleQuotationMarksPerSentence,commonlyMisspelledWordsPerSentence,functionWordsPerSentence,mostCommonWordsPerSentence
0,"In the last century, both our nations put in p...",obama,9.68,55.75,12.15,4.37,136.47,26.92,25.50,15.73,...,0,0.0,0,0,0.00,0.07,0.10,0,14.19,16.16
1,Giving all praise and honor to God. The Bible ...,obama,8.22,63.68,7.33,4.08,87.05,18.25,17.34,10.80,...,0,0.0,0,0,0.00,0.14,0.23,0,8.20,9.91
2,Just a guess. But there should be other ways p...,obama,7.89,62.56,7.85,4.27,95.34,20.23,18.37,11.33,...,0,0.0,0,0,0.02,0.03,0.59,0,9.38,11.46
3,"Mr. Speaker, Mr. Vice President, members of Co...",obama,8.65,62.79,7.74,4.42,88.56,17.91,16.55,10.53,...,0,0.0,0,0,0.05,0.01,0.29,0,8.79,10.69
4,"Well Jessica, look, I'll just give you an exam...",obama,7.23,61.80,8.52,4.10,107.25,23.49,21.34,13.67,...,0,0.0,0,0,0.05,0.02,0.76,0,12.23,13.94


# Extracting features and labels

In [22]:
def extract_features_labels(df_with_wp_indexes):
    features=df_with_wp_indexes.loc[:,"ColemanLiau":].values 
    labels=df_with_wp_indexes.loc[:, "author"].values
    return features, labels

In [23]:
x,y= extract_features_labels(speeches_with_wp)

In [24]:
print(x)

[[ 9.68 55.75 12.15 ...  0.   14.19 16.16]
 [ 8.22 63.68  7.33 ...  0.    8.2   9.91]
 [ 7.89 62.56  7.85 ...  0.    9.38 11.46]
 ...
 [ 6.68 70.    4.94 ...  0.    7.38  8.38]
 [ 6.21 68.27  5.28 ...  0.    8.82 10.26]
 [11.04 54.   12.67 ...  0.   11.32 13.42]]


In [25]:
print(y)

['obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama'
 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama'
 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama'
 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama'
 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama'
 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'obama'
 'obama' 'obama' 'obama' 'obama' 'obama' 'obama' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump' 'trump'
 'trump' 'trump' 'trump']


# Creating model and training 

In [26]:
from sklearn.svm import LinearSVC`
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=1)

In [28]:
classifier = LinearSVC(penalty="l1", dual=False, tol=1e-3)

In [29]:
classifier.fit(x_train,y_train)

/Users/niharikaghali/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
          verbose=0)

# Testing classifier

In [30]:
print(classifier.score(x_test,y_test))

0.9166666666666666


In [31]:
def test_classifier(test_string, classifier):
    print(classifier.predict([extract_indexes_names(make_one_writeprint(test_string))[0]]))

In [32]:
test_string_obama="""

Tomorrow, at this defining moment in history, you can give this country the change that we need. It starts here in Virginia. It starts here in Manassas. This is where change begins.

Our campaign has not been perfect. There are times when I look back and I've said, "you know I wouldn't have done that if I had thought about it a little bit more." But I'll tell you what. When you think about this campaign we've got a lot to be proud of when it comes to the tone that we have set.


"""
test_classifier(test_string_obama, classifier)

['obama']


In [33]:
test_string_trump="""

But it used to be that they’d argue with me, I’d fight. So I’d fight, they’d fight. I’d fight, they’d fight. Boop-boop. You’d believe me, you’d believe them. Somebody comes out. They had their point of view, I had my point of view. But you’d have an argument. Now what they do is they go silent. It’s called suppression. And that’s what happens in a communist country. That’s what they do. They suppress. You don’t fight with them anymore, unless it’s a bad. They have a little bad story about me, they’ll make it 10 times worse and it’s a major headline. But Hunter Biden, they don’t talk about him. What happened to Hunter? Where’s Hunter? Where is Hunter? They don’t talk about him.
"""
test_classifier(test_string_trump, classifier)

['trump']
